In [10]:
import requests
import pandas as pd
import tensorflow as tf
import json
import base64

In [16]:
data = pd.read_csv('dataset/clickbait.csv')

In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32000 entries, 0 to 31999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   headline   32000 non-null  object
 1   clickbait  32000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 500.1+ KB


In [18]:
data.head()

,headline,clickbait
0,Should I Get Bings,1
1,Which TV Female Friend Group Do You Belong In,1
2,"The New ""Star Wars: The Force Awakens"" Trailer...",1
3,"This Vine Of New York On ""Celebrity Big Brothe...",1
4,A Couple Did A Stunning Photo Shoot With Their...,1


In [19]:
text_test = data['headline'][1]
text_test

'Which TV Female Friend Group Do You Belong In'

In [20]:
def prepare_json(text):
    feature_spec = {
        "headline": tf.train.Feature(bytes_list=tf.train.BytesList(value=[bytes(text, "utf-8")])),
    }
    
    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()
    
    result = [
        {
            "examples": {
                "b64": base64.b64encode(example).decode()
            }
        }
    ]
    
    return json.dumps({
        "signature_name": "serving_default",
        "instances": result
    })

def make_predict(text):
    json_data = prepare_json(text)

    endpoint = "http://103.190.215.122:8501/v1/models/clickbait_prediction:predict"
    response = requests.post(endpoint, data=json_data)

    prediction = response.json().get("predictions")

    if prediction:
        prediction_value = prediction[0][0]
        result = f"{prediction_value*100.0:.1f}%: Clickbait"
    else:
        result = f"Error (endpoint tidak mereply dengan prediksi) \n {response.json()}"

    print(result)

In [21]:
make_predict(text_test)

100.0%: Clickbait


In [26]:
make_predict("Alliens will invade our earthss!")

88.2%: Clickbait


In [27]:
make_predict("Actor Discusses Challenges and Excitement in Upcoming Film Role")

17.8%: Clickbait


In [30]:
make_predict("")

Error (endpoint tidak mereply dengan prediksi) 
 {'error': 'Name: <unknown>, Feature: headline (data type: string) is required but could not be found.\n\t [[{{function_node __inference_serve_tf_examples_fn_159479}}{{node ParseExample/ParseExampleV2}}]]'}
